In [0]:
!pip install -q chakin progressbar2 textgenrnn

In [0]:
import chakin
import progressbar
import numpy as np

In [19]:
from textgenrnn import textgenrnn

textgen = textgenrnn()
textgen.generate()

Using TensorFlow backend.













[AMA Request] So I was a good sales for the end of the story of the people who want to pretty could show up the best for Americans of the Morty servers with a very way to watch some being looking for a comic passed away and I don't have to buy the most one of my shot in the woman who was a hell wi



# Scraping Lyrics from Genius

In [0]:
#Lyricsgenius has a great database of song lyrics
pip install lyricsgenius


In [0]:
#The goal here is to construct a trainng set of lyrics from a specific artist
#Rap Genius has a comprehensive database
import lyricsgenius
import re
genius = lyricsgenius.Genius("M5X2ZmPgXjw9ox14kZfoPQuQN4GJULA2RLkm7F4FK5Rch_uNLl4vIc9IqkVhxf5O")
genius.remove_section_headers = True # Remove section headers (e.g. [Chorus]) from lyrics when searching
genius.skip_non_songs = True # Include hits thought to be non-songs (e.g. track lists)
genius.excluded_terms = ["(Remix)", "(Live)"] # Exclude songs with these words in their title

In [0]:
#Choose the name of an artist and the numeber of songs to go into the training set
#More songs improves the variety of your outputs but takes longer to train
artist_name = "Taylor Swift"
n_songs = 200

In [0]:
artist = genius.search_artist(artist_name, max_songs=n_songs )

Searching for songs by Taylor Swift...

Song 1: "Lover"
Song 2: "Look What You Made Me Do"
Song 3: "End Game"
Song 4: "ME!"
Song 5: "You Need To Calm Down"
Song 6: "...Ready for It?"
Song 7: "Blank Space"
"Bad Blood (Remix)" is not valid. Skipping.
Song 8: "Call It What You Want"
Song 9: "Delicate"
"I Don’t Wanna Live Forever" is not valid. Skipping.
Song 10: "Style"
Song 11: "Gorgeous"
Song 12: "Cruel Summer"
Song 13: "Don’t Blame Me"
Song 14: "I Did Something Bad"
Song 15: "Dress"
Song 16: "The Man"
Song 17: "The Archer"
Song 18: "Bad Blood"
Song 19: "London Boy"
Song 20: "Getaway Car"
Song 21: "Miss Americana & The Heartbreak Prince"
Song 22: "King of My Heart"
Song 23: "Cornelia Street"
Song 24: "I Forgot That You Existed"
Song 25: "New Year’s Day"
Song 26: "Daylight"
Song 27: "So It Goes..."
Song 28: "This Is Why We Can’t Have Nice Things"
Song 29: "Paper Rings"
Song 30: "Death By A Thousand Cuts"
Song 31: "Wildest Dreams"
Song 32: "I Think He Knows"
Song 33: "False God"
Song 34: 

In [0]:
#Some tools for parsing

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

def strip(test_str):
    ret = ''
    skip1c = 0
    skip2c = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
    return ret

In [0]:
#Compile lyrics into a text file
lyrics = ""
sep = ':'
for song in artist.songs:
  song=str(song)
  header = song.split(sep, 1)[0]
  song_name = find_between( header, '"', '"' )
  info = genius.search_song(song_name, artist_name)
  print(info)
  add= str(info.lyrics)
  add=strip(add)
  lyrics+=add

Searching for "Lover" by Taylor Swift...
Done.
"Lover" by Taylor Swift:
    We could leave the Christmas lights up 'til January
    And this is our place, we make the rules
    And the...
Searching for "Look What You Made Me Do" by Taylor Swift...
Done.
"Look What You Made Me Do" by Taylor Swift:
    I don't like your little games
    Don't like your tilted stage
    The role you made me play
    Of the fool, n...
Searching for "End Game" by Taylor Swift...
Done.
"End Game" by Taylor Swift:
    I wanna be your end game
    I wanna be your first string
    I wanna be your A-Team (Whoa, whoa, whoa)
    I wa...
Searching for "ME!" by Taylor Swift...
Done.
"ME!" by Taylor Swift:
    I promise that you'll never find another like me
    
    I know that I'm a handful, baby, uh
    I know I never...
Searching for "You Need To Calm Down" by Taylor Swift...
Done.
"You Need To Calm Down" by Taylor Swift:
    You are somebody that I don't know
    But you're takin' shots at me like it's Patrón
  

AttributeError: ignored

In [0]:
print(lyrics[0:1000])

In [0]:
#Write out our compiled lyrics to a text file
from google.colab import files
f= open("lyrics.txt","w+")
f.write(lyrics)
files.download('lyrics.txt')
f.close

<function TextIOWrapper.close>

# Import lyrics from a text file
### If you have lyrics in a text file you can upload them from Drive to save some time

In [1]:
import requests
from google.colab import files
url = 'https://raw.github.com/Nahzuri/MigosNet/master/Lyrics/TaylorSwift_S200.txt'
response = requests.get(url)
trainingset = response.text
print(trainingset[0:1000])
f= open("trainingset.txt","w+")
f.write(trainingset)
f.close

We could leave the Christmas lights up 'til January
And this is our place, we make the rules
And there's a dazzling haze, a mysterious way about you, dear
Have I known you 20 seconds or 20 years?

Can I go where you go?
Can we always be this close?
Forever and ever, ah
Take me out, and take me home
You're my, my, my, my lover

We could let our friends crash in the living room
This is our place, we make the call
And I'm highly suspicious that everyone who sees you wants you
I've loved you three summers now, honey, but I want 'em all

Can I go where you go?
Can we always be this close?
Forever and ever, ah
Take me out, and take me home (Forever and ever)
You're my, my, my, my lover

Ladies and gentlemen, will you please stand?
With every guitar string scar on my hand
I take this magnetic force of a man to be my lover
My heart's been borrowed and yours has been blue
All's well that ends well to end up with you
Swear to be overdramatic and true to my lover
And you'll save all your dirtiest

<function TextIOWrapper.close>

# Train TextRNN on Scraped Lyrics

In [32]:
#Now our lyrics are stripped of headings and titles and saved in lyrics.text

model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 256,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 5,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 10,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 30,   # set higher to train the model for longer
    'gen_epochs': 5,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.1,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

file_name = 'trainingset.txt'
model_name = 'migos_net'


textgen = textgenrnn(name=model_name)
train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file
train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 5-layer, 256-cell LSTMs

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Training on 146,458 character sequences.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
143/143 [==============================] - 15s 104ms/step - loss: 3.8897
Epoch 2/30
143/143 [==============================] - 13s 92ms/step - loss: 3.1554
Epoch 3/30
143/143 [==============================] - 14s 98ms/step - loss: 3.1501
Epoch 4/30
143/143 [==============================] - 15s 102ms/step - loss: 3.1457
Epoch 5/30
143/143 [==============================] - 14s 97ms/step - loss: 3.1411
####################
Temperature: 0.2
####################
          a   a     o o  e                           e                                     e  t     e             e    e  s                  e     e    s           t       o     e                                              

KeyboardInterrupt: ignored

In [0]:
textgen.generate(2, temperature = 0.4)

In [0]:
textgen.generate(2, temperature = 0.9)

In [0]:
textgen.generate(prefix = "I grew up on the crime side, the New York Times side \
Stayin' alive was no jive", temperature = 0.4)

# Fine-Tune GPT2 on Scraped Lyrics

In [2]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

     |████████████████████████████████| 645kB 14.8MB/s 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 257Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 94.0Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 162Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:05, 93.2Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 160Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 121Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 154Mit/s]                                                       


In [4]:
import gpt_2_simple as gpt2
import os
import requests

model_name = "124M"
if not os.path.isdir(os.path.join("models", model_name)):
	print(f"Downloading {model_name} model...")
	gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /models/124M/


file_name = "trainingset.txt"
    

sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              file_name,
              model_name=model_name,
              steps=1000)   # steps is max number of training steps

gpt2.generate(sess)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  3.28it/s]


dataset has 51814 tokens
Training...
[1 | 10.42] loss=2.63 avg=2.63
[2 | 13.63] loss=2.35 avg=2.49
[3 | 16.85] loss=2.33 avg=2.44
[4 | 20.06] loss=2.48 avg=2.45
[5 | 23.27] loss=2.14 avg=2.39
[6 | 26.48] loss=2.14 avg=2.34
[7 | 29.68] loss=2.21 avg=2.32
[8 | 32.88] loss=2.12 avg=2.30
[9 | 36.10] loss=2.03 avg=2.27
[10 | 39.30] loss=2.13 avg=2.25
[11 | 42.51] loss=2.12 avg=2.24
[12 | 45.73] loss=2.13 avg=2.23
[13 | 48.94] loss=1.95 avg=2.21
[14 | 52.15] loss=2.00 avg=2.19
[15 | 55.36] loss=2.19 avg=2.19
[16 | 58.59] loss=1.91 avg=2.17
[17 | 61.82] loss=2.07 avg=2.17
[18 | 65.03] loss=1.74 avg=2.14
[19 | 68.25] loss=1.91 avg=2.13
[20 | 71.47] loss=1.95 avg=2.12
[21 | 74.70] loss=1.79 avg=2.10
[22 | 77.93] loss=1.90 avg=2.09
[23 | 81.15] loss=1.91 avg=2.08
[24 | 84.37] loss=1.93 avg=2.07
[25 | 87.60] loss=1.74 avg=2.06
[26 | 90.82] loss=2.05 avg=2.06
[27 | 94.05] loss=1.86 avg=2.05
[28 | 97.28] loss=1.79 avg=2.04
[29 | 100.51] loss=1.84 avg=2.03
[30 | 103.75] loss=1.81 avg=2.02
[31 | 106.

In [7]:
gpt2.mount_gdrive()
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
gpt2.generate(sess, run_name='run1')

I'm the one who makes you laugh
When you know you're 'bout to cry
And I know your favorite songs
And you tell me 'bout your dreams
Think I know where you belong
Think I know it's with me

Can't you see that I'm the one who understands you
Been here all along, so why can't you see?
You belong with me
Standing by and waiting at your back door
All this time how could you not know, baby?
You belong with me, you belong with me

Oh, I remember you driving to my house
In the middle of the night
I'm the one who makes you laugh
When you know you're 'bout to cry
And I know your favorite songs
And you tell me your favorite songs
Are you ready for it?
Comes in handy when you're in the car
And when you're not laughing, you're sleeping
It's been a long six months
And I knew you were trouble when you said
"Don't be afraid, I'm just gonna call you"

I remember you driving to my house
In the middle of the night
I'm the one who makes you laugh
When you know you're 'bout to cry
And I know your favorite s

In [9]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="I grew up on the crime side, the New York Times side \
Stayin' alive was no jive",
              nsamples=5,
              batch_size=5
              )

I grew up on the crime side, the New York Times side Stayin' alive was no jive, no fix, no na come on, it's all me, it's all me
I'm with you, I'm with you
(I know I'll never get far, I know I'll never get far)
I'm with you, I'm with you
(I know I'll never get far, I know I'll never get far)

Like any great love, it keeps you guessing
And I remember
What you wanted, I changed
And I remember
You said, "I'll never let you go, I'll never let you go"
But you were still a boy, like any good boy
He got his perfect daydream, he made his own
He was always dreaming, and I was just as happy
As I was when he woke
I was with him, I was with him
He was with him, I was with him

I want you like a best friend
I want you like a best friend
(And if you're anything like me, I'm really, really, really, really)
So if you're anything like me, I'm really, really, really, really
(And if you're anything like me
I grew up on the crime side, the New York Times side Stayin' alive was no jive, no meantime, no mean

In [16]:
gpt2.generate(sess,
              length=100,
              temperature=0.9,
              prefix="New York",
              nsamples=10,
              batch_size=1
              )

New York? New York?
It's so sad to
Think about the good times
You and I

’Cause baby, now we've got bad blood
You know it used to be mad love
So take a look what you've done
’Cause baby, now we've got bad blood, hey!
Now we've got problems
And I don't think we can solve 'em
You made a really deep cut
And baby, now we've got bad blood
New York (It's a sideshow)
But New York (Oh, oh, oh, oh, oh)
Is haunted
Don't you think I was too young to be messed with
The girl in the dress cried the whole way home

And I could write new rules for you
And break them down into small pieces, but that's impossible
When I'm not writing rules for you
And I can't do it with you

Saying goodbye is death by a thousand
New York, that's my whole town
They whisper in the hallway, "She's a bad, bad girl" (Okay)
When she's not running, she's combative, saying stupid things
But when she's with you, she's so mean
I can't help thinking
These are the hands of fate
You're gonna have to get me out, but you'll get me

An

# Train TextRNN on a Second Training Set to Write a Duet (in progress)

In [20]:
url = 'https://raw.github.com/Nahzuri/MigosNet/master/Lyrics/Kanye_S200.txt'
response = requests.get(url)
trainingset = response.text
print(trainingset[0:1000])
f= open("trainingset_2.txt","w+")
f.write(trainingset)
f.close

Well, it is a weepin' and a moanin' and a gnashin' of teeth
It is a weepin' and a mournin' and a gnashin' of teeth
It is a—when it comes to my sound which is the champion sound
Believe, believe

O-o-o-o-o-okay, Lamborghini Mercy
Your chick, she so thirsty
I-I-I-I-I'm in that two-seat Lambo
With your girl, she tryna jerk me (Believe)
O-o-o-o-o-okay, Lamborghini Mercy
Your chick, she so thirsty
I-I-I-I-I'm in that two-seat Lambo
With your girl, she tryna jerk me
O-o-o-o-o-okay, Lamborghini Mercy (Swerve)
Your chick, she so thirsty (Swerve)
I-I-I-I-I'm in that two-seat Lambo
With your girl, she tryna jerk me (Woah, believe)
O-o-o-o-o-okay, Lamborghini Mercy
Your chick, she so thirsty (Boy)
I-I-I-I-I'm in that two-seat Lambo (Boy)
With your girl, she tryna jerk me

Okay, drop it to the floor, make that ass shake (Shake, shake)
Woah, make the ground move: that's an ass quake
Built a house up on that ass: that's an ass-state
Roll–roll–roll my weed on it: that's an ass tray
Say, Ye, say, Ye, 

<function TextIOWrapper.close>

In [21]:


model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 256,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 5,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 10,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 30,   # set higher to train the model for longer
    'gen_epochs': 5,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.1,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

file_name = 'trainingset_2.txt'
model_name = 'migos_net'


textgen = textgenrnn(name=model_name)
train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file
train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 5-layer, 256-cell LSTMs

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Training on 330,925 character sequences.


Epoch 1/30
323/323 [==============================] - 39s 121ms/step - loss: 3.2660
Epoch 2/30
323/323 [==============================] - 27s 83ms/step - loss: 3.2145
Epoch 3/30
323/323 [==============================] - 27s 85ms/step - loss: 3.2052
Epoch 4/30
323/323 [==============================] - 28s 85ms/step - loss: 2.7976
Epoch 5/30
323/323 [==============================] - 28s 85ms/step - loss: 1.8734
####################
Temperature: 0.2
####################
he crack and what it all the can should in the crands to be the tried the brobe the could up in the brother what it what it all the fame in the chain the will not the crazy that is a god a money and the child in the crazy the crazy now in the chard of the really the brother and the chartions the fam

not the start
I go

In [23]:
textgen.generate(prefix = "Are you there?", temperature = 0.8)

ere?
Now I spent another models
But how many ladies tall me for the team
G.O.O.D. Fridays
I'ma take, I'ma take, I'ma take this one for the team (Hahahahaha
You know what I’m about
Reapenicn to sleep with no make-up you a fig' how I got class
Uh, 3 Shaty Studla
Yeah I always had a fast ass should hav



In [40]:
gptout_1 = str(gpt2.generate(sess,
              length=150,
              temperature=0.7,
              prefix="No He Didn't (Yes I Did)",
              nsamples=10,
              batch_size=1
              ))

No He Didn't (Yes I Did)High above the rest, in the silence
I forgot that I didn't need to
Talk to them anymore
So many names started coming and going
But you wouldn't spell them, no

I don't want you to
Just thought you should know
That I don't want you to
Just thought you should know
That I don't want you to
Just thought you should know
That I don't want you to
Just thought you should know
That I don't want you to
Just thought you should know
That I don't want you to
Just thought you should know
That I don't want you to
Just thought you should know
That I don't want you to
Just thought you should
No He Didn't (Yes I Did)
I'm so sick of running as fast as I can
Wondering if I'd get there quicker if I was a man (Whoa, whoa, whoa)
And I'm so sick of them coming at me again (Coming at me again)
'Cause if I was a man (If I was a man)
Then I'd be the man (Then I'd be the man)
I'm so sick of running as fast as I can (As fast as I can)
Wondering if I'd get there quicker if I was a man (Hey)


In [42]:
print(gptout_1)

None


In [35]:
textgen.generate(prefix = "instring", temperature = 1)

nd Mone Gayer
Yeah, baby, no more parties in L.A., uh
No more parties in L.A., uh
No more parties in L.A., uh
No more parties in L.A., uh
No more (Lyin', Li coopr tickets, this is more than I love you more
Let's call out names, names, for sure

I can't love you up in the Degty all the child, I know 



In [0]:
instring = """I knict the magic hour
And outta album and I like to leave
You can't believe me
And I know she’s lookin' like six weeks
You left your rab 'cause they'd know what I'm about to do
(Hey Mama)
"""

In [39]:
gpt2.generate(sess,
              length=150,
              temperature=0.7,
              prefix=instring,
              nsamples=10,
              batch_size=1
              )

I knict the magic hour
And outta album and I like to leave
You can't believe me
And I know she’s lookin' like six weeks
You left your rab 'cause they'd know what I'm about to do
(Hey Mama)
I'm the one who makes you laugh, not your typical sad, sad, sad
You make me cry, and when I laugh, it's because it's so wrong
You make me cry harder, yeah, I know
That's what you do, baby, you make me cry
(Ooh, ooh, oh, oh, oh)

Ooh, ooh, ooh, ooh, oh, oh

You made me laugh, and when I'm about to lose it
You made me cry, and when I'm about to win it
You made me smile, and when I'm about to lose it
You made me laugh, and when I'm about to win it
You made
I knict the magic hour
And outta album and I like to leave
You can't believe me
And I know she’s lookin' like six weeks
You left your rab 'cause they'd know what I'm about to do
(Hey Mama)

Maybe I went psycho right there
But it's not like Irrly things we do
Or the way she talks
Or the way I stare at you
Or the way this song
Swayes you like a bad drea